In [1]:
import numpy as np
import scipy.optimize as op
import pandas as pd
import os
import glob
from multiprocessing import Pool

import emcee
import uvlf

In [ ]:
nwalkers,nsteps=50,22000

In [18]:
DM='WDM'
fs_model='DPL'
dataset_name='B'
z_arr=[6]
    
#priors:
log10_ks_p1=-2.
log10_ks_p2=1.

a_s1_p1=-1.
a_s1_p2=1.

a_s2_p1=-1.
a_s2_p2=1.


log10_M_с_p1=8.
log10_M_с_p2=10

log10_M_t_p1=8.
log10_M_t_p2=10

mx_p1=0.5
mx_p2=10.

In [19]:
ind=1

if fs_model=='DPLM':
    ind=-1
    


ndim=3

if DM=='WDM':
    ndim+=1
if 'DPL' in fs_model:
    ndim+=2


In [4]:
LF_data=uvlf.dataset(dataset_name,z_arr)
const=0
for z,lf in LF_data.items():
        x=lf[0]
        y=lf[1]
        yerr_down=lf[3]
        yerr_up=lf[2]
        const+=-0.5*len(y)*np.log((np.pi/2))-sum(np.log(yerr_down+yerr_up))


In [5]:
def chi2(y_pred,y, yerr_up, yerr_down):
        
    return np.sum(np.heaviside(y_pred-y,0)*((y_pred-y)/yerr_up)**2+np.heaviside(y-y_pred,0)*((y_pred-y)/yerr_down)**2)

def lnlike(theta, LF_data=LF_data):
    log10_ks,a_s1,a_s2,log10_M_c,log10_M_t,mx = theta
        
    M_t=10**log10_M_t
    M_c=10**log10_M_c
    ks=10**log10_ks
    chi2_sum=0
    for z,lf in LF_data.items():
        x=lf[0]
        y=lf[1]
        yerr_down=lf[3]
        yerr_up=lf[2]
        
        
        
        M_UV_Model, phi_Model = uvlf.compute_LF(z=z, ks=ks,a_s1=a_s1, a_s2=a_s2,M_с=M_c,M_t=M_t,DM=DM,mx=mx, ind=ind)
    
        y_pred = np.interp( x, M_UV_Model[::-1], np.log10(phi_Model)[::-1] )
#        print(chi2(y_interp,y,yerr_up,yerr_down))
        chi2_sum+=chi2(y_pred,y,yerr_up,yerr_down)
        
    return -0.5*chi2_sum+const

In [ ]:
nll = lambda *args: -lnlike(*args)


def lnprior(theta):
    
    log10_ks,a_s1,a_s2,log10_M_с,log10_M_t,mx = theta
    if log10_ks_p1<log10_ks<log10_ks_p2  and  a_s1_p1 < a_s1 < a_s1_p2  and  a_s2_p1<a_s2<a_s2_p2  and log10_M_с_p1<log10_M_с<log10_M_с_p2 and log10_M_t_p1<log10_M_t<log10_M_t_p2 and mx_p1<mx<mx_p2:


        return 0.0
    return -np.inf


def lnprob(theta, LF_data=LF_data):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    ll=lnlike(theta, LF_data)

    if np.isnan(ll):
        return -np.inf
    return ll+lp

In [7]:
a_s1_True=0.25
a_s2_True=0.25

log10M_t_True = 8.5
log10M_c_True = 8.5
log10_ks_True=-0.99
mx_True=3.5
result = op.minimize(nll, [log10_ks_True,a_s1_True,a_s2_True,log10M_c_True, log10M_t_True, mx_True],args=(LF_data),method='TNC',bounds=((-2,1),(-0.49,0.49),(-0.49,0.49),(8.1,11),(8.1,11),(0.5,5)))

'sample_wdm_B_z6_nw_50_nst_22000_pl.h5'

In [ ]:
filename = "sample_{}_{}_z{}_nw_{}_nst_{}_{}.h5".format(DM.lower(),dataset_name,'-'.join(str(x) for x in z_arr), nwalkers,nsteps,fs_model.lower())

pos = [result['x']+1e-3*np.random.randn(ndim) for i in range(nwalkers)]
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, ndim)
with Pool(processes=8) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(LF_data,), pool=pool,backend=backend)
    sampler.run_mcmc(initial_state=pos, nsteps=nsteps,progress=True,store = True)